### 初始化配置

In [1]:
import torch
torch.cuda.empty_cache()
# 修改这三个参数，以及下面代码中所有的文件地址
device = ''
model_path = ""
model_name = ""

### 可能需要在ppl_and_ngram当中需要修改的部分
1. load_model中根据待测模型特性决定是否添加单独if语句
2. load_data_from_jsonl中根据待测数据集，修改样本format方式
3. calculate_answer_ppl中根据tokenizer特性决定是否要用' '+sep_token（123行）：可以先运行 如果有很多Not found报错，可修改为另一个前面不加' '的
4. calculate_n_gram_accuracy如果运行出错，可以从所有跟if相关的语句开始查，为当前模型添加到if条件中或者写一个新的if

### 测合成数据

In [ ]:
from ppl_and_ngram import *


# 数据集路径
dataset_names = [
                "GSM8K_rewritten-test-1",
                "GSM8K_rewritten-test-2",
                "GSM8K_rewritten-test-3",
                "GSM8K_rewritten-train-1",
                "GSM8K_rewritten-train-2",
                "GSM8K_rewritten-train-3",
                "MATH_rewritten-test-1",
                "MATH_rewritten-test-2",
                "MATH_rewritten-test-3",
                "MATH_rewritten-train-1",
                "MATH_rewritten-train-2",
                "MATH_rewritten-train-3"
                ]

model, tokenizer = load_model(model_path, device)

results_5grm_summary = {}
results_10grm_summary = {}

k = 5  # 你的 starting points 数量

# 遍历每个数据集并执行测试
for dataset_name in dataset_names:
    # 加载数据集
    dataset_path = f'/data/rjxu/MATH&GSM8K/{dataset_name}.jsonl'
    dataset = load_data_from_jsonl(dataset_path)
    
    # 执行计算
    output_file_5gram = f'/home/rjxu/rewritten/ngram/5gram-{model_name}-{dataset_name}.jsonl'
    ngram_5_results = calculate_n_gram_accuracy(5, k, dataset, model, tokenizer, device, output_file_5gram)
    print(f"{dataset_name} 5_gram_accuracy: ", ngram_5_results["mean_n_grams"])
    results_5grm_summary[f'{dataset_name}'] = ngram_5_results["mean_n_grams"]

    output_file_10gram = f'/home/rjxu/rewritten/ngram/10gram-{model_name}-{dataset_name}.jsonl'
    ngram_10_results = calculate_n_gram_accuracy(10, k, dataset, model, tokenizer, device, output_file_10gram)
    print(f"{dataset_name} 10_gram_accuracy: ", ngram_10_results["mean_n_grams"])
    results_10grm_summary[f'{dataset_name}'] = ngram_10_results["mean_n_grams"]

# 打印结果
for key, value in results_5grm_summary.items():
    print(f"{key}: {value}")
print("5gram")
print("| GSM8K-train-1 | GSM8K-train-2 | GSM8K-train-3 | GSM8K-test-1 | GSM8K-test-2 | GSM8K-test-3 | MATH-train-1 | MATH-train-2 | MATH-train-3 | MATH-test-1 | MATH-test-2 | MATH-test-3 |")
print("|---------------|---------------|---------------|--------------|--------------|--------------|--------------|--------------|--------------|-------------|-------------|-------------|")
print(f"| {results_5grm_summary['GSM8K_rewritten-train-1'] * 100:.2f} | {results_5grm_summary['GSM8K_rewritten-train-2'] * 100:.2f} | {results_5grm_summary['GSM8K_rewritten-train-3'] * 100:.2f} | "
      f"{results_5grm_summary['GSM8K_rewritten-test-1'] * 100:.2f} | {results_5grm_summary['GSM8K_rewritten-test-2'] * 100:.2f} | {results_5grm_summary['GSM8K_rewritten-test-3'] * 100:.2f} | "
      f"{results_5grm_summary['MATH_rewritten-train-1'] * 100:.2f} | {results_5grm_summary['MATH_rewritten-train-2'] * 100:.2f} | {results_5grm_summary['MATH_rewritten-train-3'] * 100:.2f} | "
      f"{results_5grm_summary['MATH_rewritten-test-1'] * 100:.2f} | {results_5grm_summary['MATH_rewritten-test-2'] * 100:.2f} | {results_5grm_summary['MATH_rewritten-test-3'] * 100:.2f} |")

for key, value in results_10grm_summary.items():
    print(f"{key}: {value}")
print("10gram")
print("| GSM8K-train-1 | GSM8K-train-2 | GSM8K-train-3 | GSM8K-test-1 | GSM8K-test-2 | GSM8K-test-3 | MATH-train-1 | MATH-train-2 | MATH-train-3 | MATH-test-1 | MATH-test-2 | MATH-test-3 |")
print("|---------------|---------------|---------------|--------------|--------------|--------------|--------------|--------------|--------------|-------------|-------------|-------------|")
print(f"| {results_10grm_summary['GSM8K_rewritten-train-1'] * 100:.2f} | {results_10grm_summary['GSM8K_rewritten-train-2'] * 100:.2f} | {results_10grm_summary['GSM8K_rewritten-train-3'] * 100:.2f} | "
      f"{results_10grm_summary['GSM8K_rewritten-test-1'] * 100:.2f} | {results_10grm_summary['GSM8K_rewritten-test-2'] * 100:.2f} | {results_10grm_summary['GSM8K_rewritten-test-3'] * 100:.2f} | "
      f"{results_10grm_summary['MATH_rewritten-train-1'] * 100:.2f} | {results_10grm_summary['MATH_rewritten-train-2'] * 100:.2f} | {results_10grm_summary['MATH_rewritten-train-3'] * 100:.2f} | "
      f"{results_10grm_summary['MATH_rewritten-test-1'] * 100:.2f} | {results_10grm_summary['MATH_rewritten-test-2'] * 100:.2f} | {results_10grm_summary['MATH_rewritten-test-3'] * 100:.2f} |")



### 测5gram原数据（含rouge_l & edit_distance统计）

In [3]:
from calculate_similarity_metrics import *
from ppl_and_ngram import *

# 数据集路径
dataset_names = [
                 "MATH-test",
                 "MATH-train",
                 "GSM8K-test",
                 "GSM8K-train"
                ]

# 设置你的参数
k = 5  # 你的 starting points 数量

results_5grm_summary = {}
model, tokenizer = load_model(model_path, device)

# 遍历每个数据集并执行测试
for dataset_name in dataset_names:
    # 加载数据集
    dataset_path = f'/home/rjxu/new-gram/datasets/{dataset_name}.jsonl'
    dataset = load_data_from_jsonl(dataset_path)

    path = f"/home/rjxu/new-gram/results/5gram-{model_name}-{dataset_name}.json"

    ngram_5_results = calculate_similarity_metrics(5, k, dataset, model, tokenizer, device, path)
    print(f"{dataset_name} 5_gram_accuracy: ")
    print(f"Mean Accuracy Edit Distance: {ngram_5_results['mean_accuracy_edit_distance']}")
    print(f"Mean Accuracy Rouge-L: {ngram_5_results['mean_accuracy_rouge_l']}")
    print(f"Mean Accuracy: {ngram_5_results['mean_accuracy']}")
    # 存储结果到字典中
    results_5grm_summary[f'{dataset_name}'] = ngram_5_results

# 输出 Markdown 格式表格
print("Edit Distance")
print("| GSM8K-train | GSM8K-test | MATH-train | MATH-test |")
print("|-------------|------------|------------|-----------|")

print(f"| {results_5grm_summary['GSM8K-train']['mean_accuracy_edit_distance'] * 100:.2f} | "
      f"{results_5grm_summary['GSM8K-test']['mean_accuracy_edit_distance'] * 100:.2f} | "
      f"{results_5grm_summary['MATH-train']['mean_accuracy_edit_distance'] * 100:.2f} | "
      f"{results_5grm_summary['MATH-test']['mean_accuracy_edit_distance'] * 100:.2f} |")

# 输出 Markdown 格式表格
print("Rouge-L")
print("| GSM8K-train | GSM8K-test | MATH-train | MATH-test |")
print("|-------------|------------|------------|-----------|")

print(f"| {results_5grm_summary['GSM8K-train']['mean_accuracy_rouge_l'] * 100:.2f} | "
      f"{results_5grm_summary['GSM8K-test']['mean_accuracy_rouge_l'] * 100:.2f} | "
      f"{results_5grm_summary['MATH-train']['mean_accuracy_rouge_l'] * 100:.2f} | "
      f"{results_5grm_summary['MATH-test']['mean_accuracy_rouge_l'] * 100:.2f} |")

# 输出 Markdown 格式表格
print("n-gram")
print("| GSM8K-train | GSM8K-test | MATH-train | MATH-test |")
print("|-------------|------------|------------|-----------|")

print(f"| {results_5grm_summary['GSM8K-train']['mean_accuracy'] * 100:.2f} | "
      f"{results_5grm_summary['GSM8K-test']['mean_accuracy'] * 100:.2f} | "
      f"{results_5grm_summary['MATH-train']['mean_accuracy'] * 100:.2f} | "
      f"{results_5grm_summary['MATH-test']['mean_accuracy'] * 100:.2f} |")

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

3000


100%|██████████| 3000/3000 [30:32<00:00,  1.64it/s]


MATH-test 5_gram_accuracy: 
Mean Accuracy Edit Distance: 0.3546000000000001
Mean Accuracy Rouge-L: 0.4256
Mean Accuracy: 0.33026666666666665
3000


...

MATH-train 5_gram_accuracy: 
Mean Accuracy Edit Distance: 0.36133333333333334
Mean Accuracy Rouge-L: 0.4344
Mean Accuracy: 0.3390000000000001
1319


100%|██████████| 1319/1319 [12:22<00:00,  1.78it/s]


GSM8K-test 5_gram_accuracy: 
Mean Accuracy Edit Distance: 0.20833965125094767
Mean Accuracy Rouge-L: 0.26974981046247154
Mean Accuracy: 0.19393479909021985
3000


100%|██████████| 3000/3000 [28:09<00:00,  1.78it/s]

GSM8K-train 5_gram_accuracy: 
Mean Accuracy Edit Distance: 0.20140000000000002
Mean Accuracy Rouge-L: 0.26566666666666666
Mean Accuracy: 0.1875333333333333
Edit Distance
| GSM8K-train | GSM8K-test | MATH-train | MATH-test |
|-------------|------------|------------|-----------|
| 20.14 | 20.83 | 36.13 | 35.46 |
Rouge-L
| GSM8K-train | GSM8K-test | MATH-train | MATH-test |
|-------------|------------|------------|-----------|
| 26.57 | 26.97 | 43.44 | 42.56 |
n-gram
| GSM8K-train | GSM8K-test | MATH-train | MATH-test |
|-------------|------------|------------|-----------|
| 18.75 | 19.39 | 33.90 | 33.03 |


### 测10gram原数据（含rouge_l & edit_distance统计）

In [4]:
from calculate_similarity_metrics import *
from ppl_and_ngram import *

# 数据集路径
dataset_names = [
                 "MATH-test",
                 "MATH-train",
                 "GSM8K-test",
                 "GSM8K-train"
                ]

# 设置你的参数
k = 5  # 你的 starting points 数量

results_10grm_summary = {}

# 遍历每个数据集并执行测试
for dataset_name in dataset_names:
    # 加载数据集
    dataset_path = f'/home/rjxu/new-gram/datasets/{dataset_name}.jsonl'
    dataset = load_data_from_jsonl(dataset_path)

    path = f"/home/rjxu/new-gram/results/10gram-{model_name}-{dataset_name}.json"

    ngram_10_results = calculate_similarity_metrics(10, k, dataset, model, tokenizer, device, path)
    print(f"{dataset_name} 10_gram_accuracy: ")
    print(f"Mean Accuracy Edit Distance: {ngram_10_results['mean_accuracy_edit_distance']}")
    print(f"Mean Accuracy Rouge-L: {ngram_10_results['mean_accuracy_rouge_l']}")
    print(f"Mean Accuracy: {ngram_10_results['mean_accuracy']}")
    # 存储结果到字典中
    results_10grm_summary[f'{dataset_name}'] = ngram_5_results

# 输出 Markdown 格式表格
print("Edit Distance")
print("| GSM8K-train | GSM8K-test | MATH-train | MATH-test |")
print("|-------------|------------|------------|-----------|")

print(f"| {results_10grm_summary['GSM8K-train']['mean_accuracy_edit_distance'] * 100:.2f} | "
      f"{results_10grm_summary['GSM8K-test']['mean_accuracy_edit_distance'] * 100:.2f} | "
      f"{results_10grm_summary['MATH-train']['mean_accuracy_edit_distance'] * 100:.2f} | "
      f"{results_10grm_summary['MATH-test']['mean_accuracy_edit_distance'] * 100:.2f} |")

# 输出 Markdown 格式表格
print("Rouge-L")
print("| GSM8K-train | GSM8K-test | MATH-train | MATH-test |")
print("|-------------|------------|------------|-----------|")

print(f"| {results_10grm_summary['GSM8K-train']['mean_accuracy_rouge_l'] * 100:.2f} | "
      f"{results_10grm_summary['GSM8K-test']['mean_accuracy_rouge_l'] * 100:.2f} | "
      f"{results_10grm_summary['MATH-train']['mean_accuracy_rouge_l'] * 100:.2f} | "
      f"{results_10grm_summary['MATH-test']['mean_accuracy_rouge_l'] * 100:.2f} |")

# 输出 Markdown 格式表格
print("n-gram")
print("| GSM8K-train | GSM8K-test | MATH-train | MATH-test |")
print("|-------------|------------|------------|-----------|")

print(f"| {results_10grm_summary['GSM8K-train']['mean_accuracy'] * 100:.2f} | "
      f"{results_10grm_summary['GSM8K-test']['mean_accuracy'] * 100:.2f} | "
      f"{results_10grm_summary['MATH-train']['mean_accuracy'] * 100:.2f} | "
      f"{results_10grm_summary['MATH-test']['mean_accuracy'] * 100:.2f} |")

3000


100%|██████████| 3000/3000 [57:33<00:00,  1.15s/it]


MATH-test 10_gram_accuracy: 
Mean Accuracy Edit Distance: 0.17393333333333333
Mean Accuracy Rouge-L: 0.305
Mean Accuracy: 0.13666666666666666
3000


100%|██████████| 3000/3000 [57:46<00:00,  1.16s/it] 


MATH-train 10_gram_accuracy: 
Mean Accuracy Edit Distance: 0.18026666666666666
Mean Accuracy Rouge-L: 0.303
Mean Accuracy: 0.14366666666666666
1319


100%|██████████| 1319/1319 [24:23<00:00,  1.11s/it]


GSM8K-test 10_gram_accuracy: 
Mean Accuracy Edit Distance: 0.12160727824109174
Mean Accuracy Rouge-L: 0.20181956027293405
Mean Accuracy: 0.10326004548900682
3000


100%|██████████| 3000/3000 [56:01<00:00,  1.12s/it]

GSM8K-train 10_gram_accuracy: 
Mean Accuracy Edit Distance: 0.11646666666666666
Mean Accuracy Rouge-L: 0.19893333333333335
Mean Accuracy: 0.09753333333333335
Edit Distance
| GSM8K-train | GSM8K-test | MATH-train | MATH-test |
|-------------|------------|------------|-----------|
| 20.14 | 20.14 | 20.14 | 20.14 |
Rouge-L
| GSM8K-train | GSM8K-test | MATH-train | MATH-test |
|-------------|------------|------------|-----------|
| 26.57 | 26.57 | 26.57 | 26.57 |
n-gram
| GSM8K-train | GSM8K-test | MATH-train | MATH-test |
|-------------|------------|------------|-----------|
| 18.75 | 18.75 | 18.75 | 18.75 |


### 测ppl合成数据

In [2]:
from ppl_and_ngram import *


# 数据集路径
dataset_names = [
                "GSM8K_rewritten-test-1",
                "GSM8K_rewritten-test-2",
                "GSM8K_rewritten-test-3",
                "GSM8K_rewritten-train-1",
                "GSM8K_rewritten-train-2",
                "GSM8K_rewritten-train-3",
                "MATH_rewritten-test-1",
                "MATH_rewritten-test-2",
                "MATH_rewritten-test-3",
                "MATH_rewritten-train-1",
                "MATH_rewritten-train-2",
                "MATH_rewritten-train-3"
                ]

model, tokenizer = load_model(model_path, device)

results_ppl_summary = {}

# 遍历每个数据集并执行测试
for dataset_name in dataset_names:
    # 加载数据集
    dataset_path = f'/data/rjxu/MATH&GSM8K/{dataset_name}.jsonl'
    dataset = load_data_from_jsonl(dataset_path)

    # 执行计算
    output_file_ppl = f'/home/rjxu/rewritten/ppl/ppl-{model_name}-{dataset_name}.jsonl'
    ppl_results = calculate_answer_ppl(dataset, model, tokenizer, device, output_file_ppl)
    print(f"{dataset_name} Average_ppl_accuracy: ", ppl_results["mean_perplexity"])
    results_ppl_summary[f'{dataset_name}'] = ppl_results["mean_perplexity"]

for key, value in results_ppl_summary.items():
    print(f"{key}: {value}")
print("ppl")
print("| GSM8K-train-1 | GSM8K-train-2 | GSM8K-train-3 | GSM8K-test-1 | GSM8K-test-2 | GSM8K-test-3 | MATH-train-1 | MATH-train-2 | MATH-train-3 | MATH-test-1 | MATH-test-2 | MATH-test-3 |")
print("|---------------|---------------|---------------|--------------|--------------|--------------|--------------|--------------|--------------|-------------|-------------|-------------|")
print(f"| {results_ppl_summary['GSM8K_rewritten-train-1']:.2f} | {results_ppl_summary['GSM8K_rewritten-train-2']:.2f} | {results_ppl_summary['GSM8K_rewritten-train-3']:.2f} | "
      f"{results_ppl_summary['GSM8K_rewritten-test-1']:.2f} | {results_ppl_summary['GSM8K_rewritten-test-2']:.2f} | {results_ppl_summary['GSM8K_rewritten-test-3']:.2f} | "
      f"{results_ppl_summary['MATH_rewritten-train-1']:.2f} | {results_ppl_summary['MATH_rewritten-train-2']:.2f} | {results_ppl_summary['MATH_rewritten-train-3']:.2f} | "
      f"{results_ppl_summary['MATH_rewritten-test-1']:.2f} | {results_ppl_summary['MATH_rewritten-test-2']:.2f} | {results_ppl_summary['MATH_rewritten-test-3']:.2f} |")

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

1319


100%|██████████| 1319/1319 [08:38<00:00,  2.55it/s]


GSM8K_rewritten-test-1 Average_ppl_accuracy:  3.0913105150169273
1319


...

MATH_rewritten-train-1 Average_ppl_accuracy:  2.858447554429372
3000


100%|██████████| 3000/3000 [20:31<00:00,  2.44it/s]


MATH_rewritten-train-2 Average_ppl_accuracy:  2.7421350757280987
3000


100%|██████████| 3000/3000 [20:30<00:00,  2.44it/s]

MATH_rewritten-train-3 Average_ppl_accuracy:  2.7448879845142367
GSM8K_rewritten-test-1: 3.0913105150169273
GSM8K_rewritten-test-2: 3.1224512857772617
GSM8K_rewritten-test-3: 3.0890247505482984
GSM8K_rewritten-train-1: 3.0962917334636053
GSM8K_rewritten-train-2: 3.100403634428978
GSM8K_rewritten-train-3: 3.1042084426085155
MATH_rewritten-test-1: 2.821520067413648
MATH_rewritten-test-2: 2.718368011275927
MATH_rewritten-test-3: 2.7075223783651987
MATH_rewritten-train-1: 2.858447554429372
MATH_rewritten-train-2: 2.7421350757280987
MATH_rewritten-train-3: 2.7448879845142367
ppl
| GSM8K-train-1 | GSM8K-train-2 | GSM8K-train-3 | GSM8K-test-1 | GSM8K-test-2 | GSM8K-test-3 | MATH-train-1 | MATH-train-2 | MATH-train-3 | MATH-test-1 | MATH-test-2 | MATH-test-3 |
|---------------|---------------|---------------|--------------|--------------|--------------|--------------|--------------|--------------|-------------|-------------|-------------|
| 3.10 | 3.10 | 3.10 | 3.09 | 3.12 | 3.09 | 2.86 | 2.74

### 测ppl原数据

In [3]:
dataset_names = [
                 "MATH-test",
                 "MATH-train",
                 "GSM8K-test",
                 "GSM8K-train"
                ]

# model, tokenizer = load_model(model_path, device)

results_ppl_summary = {}

# 遍历每个数据集并执行测试
for dataset_name in dataset_names:
    # 加载数据集
    dataset_path = f'/home/rjxu/new-gram/datasets/{dataset_name}.jsonl'
    dataset = load_data_from_jsonl(dataset_path)

    # 执行计算
    output_file_ppl = f'/home/rjxu/rewritten/ppl/ppl-{model_name}-{dataset_name}.jsonl'
    ppl_results = calculate_answer_ppl(dataset, model, tokenizer, device, output_file_ppl)
    print(f"{dataset_name} Average_ppl_accuracy: ", ppl_results["mean_perplexity"])
    results_ppl_summary[f'{dataset_name}'] = ppl_results["mean_perplexity"]

for key, value in results_ppl_summary.items():
    print(f"{key}: {value}")
print("ppl")
print("| GSM8K-train | GSM8K-test | MATH-train | MATH-test |")
print("|-------------|------------|------------|-----------|")
print(f"| {results_ppl_summary['GSM8K-train-']:.2f} | "
      f"{results_ppl_summary['GSM8K-test']:.2f} | "
      f"{results_ppl_summary['MATH-train']:.2f} | "
      f"{results_ppl_summary['MATH-test']:.2f}")

3000


100%|██████████| 3000/3000 [20:19<00:00,  2.46it/s]


MATH-test Average_ppl_accuracy:  2.4194161751270293
3000


100%|██████████| 3000/3000 [20:30<00:00,  2.44it/s]


MATH-train Average_ppl_accuracy:  2.457964661359787
1319


100%|██████████| 1319/1319 [08:39<00:00,  2.54it/s]


GSM8K-test Average_ppl_accuracy:  2.9528017035933316
3000


100%|██████████| 3000/3000 [19:38<00:00,  2.55it/s]

GSM8K-train Average_ppl_accuracy:  2.9656837712526323
MATH-test: 2.4194161751270293
MATH-train: 2.457964661359787
GSM8K-test: 2.9528017035933316
GSM8K-train: 2.9656837712526323
ppl
| GSM8K-train | GSM8K-test | MATH-train | MATH-test |
|-------------|------------|------------|-----------|


KeyError: 'GSM8K-train-'

In [4]:
print("| GSM8K-train | GSM8K-test | MATH-train | MATH-test |")
print("|-------------|------------|------------|-----------|")
print(f"| {results_ppl_summary['GSM8K-train']:.2f} | "
      f"{results_ppl_summary['GSM8K-test']:.2f} | "
      f"{results_ppl_summary['MATH-train']:.2f} | "
      f"{results_ppl_summary['MATH-test']:.2f} |")

| GSM8K-train | GSM8K-test | MATH-train | MATH-test |
|-------------|------------|------------|-----------|
| 2.97 | 2.95 | 2.46 | 2.42 |


### 如果测试过程中断，可以通过复制日志输出将已有记录存入字典，继续测试

In [1]:
import re

# 假设这是从日志文件中读取的内容
log_data = """
GSM8K_rewritten-test-1 5_gram_accuracy:  0.002577710386656558
no special token
100%|██████████| 1319/1319 [08:17<00:00,  2.65it/s]
Using eos_token, but it is not set yet.
GSM8K_rewritten-test-1 10_gram_accuracy:  0.0
1319
no special token
100%|██████████| 1319/1319 [04:28<00:00,  4.91it/s]
Using eos_token, but it is not set yet.
GSM8K_rewritten-test-2 5_gram_accuracy:  0.0027293404094010615
no special token
100%|██████████| 1319/1319 [08:14<00:00,  2.66it/s]
Using eos_token, but it is not set yet.
GSM8K_rewritten-test-2 10_gram_accuracy:  0.0
1319
no special token
100%|██████████| 1319/1319 [04:29<00:00,  4.89it/s]
Using eos_token, but it is not set yet.
GSM8K_rewritten-test-3 5_gram_accuracy:  0.003032600454890068
no special token
100%|██████████| 1319/1319 [08:21<00:00,  2.63it/s]
Using eos_token, but it is not set yet.
GSM8K_rewritten-test-3 10_gram_accuracy:  0.0
3000
no special token
100%|██████████| 3000/3000 [10:16<00:00,  4.86it/s]
Using eos_token, but it is not set yet.
GSM8K_rewritten-train-1 5_gram_accuracy:  0.0019333333333333336
no special token
100%|██████████| 3000/3000 [19:00<00:00,  2.63it/s]
Using eos_token, but it is not set yet.
GSM8K_rewritten-train-1 10_gram_accuracy:  0.0
3000
no special token
100%|██████████| 3000/3000 [10:14<00:00,  4.89it/s]
Using eos_token, but it is not set yet.
GSM8K_rewritten-train-2 5_gram_accuracy:  0.002266666666666667
no special token
100%|██████████| 3000/3000 [18:43<00:00,  2.67it/s]
Using eos_token, but it is not set yet.
GSM8K_rewritten-train-2 10_gram_accuracy:  6.666666666666667e-05
3000
no special token
100%|██████████| 3000/3000 [10:14<00:00,  4.88it/s]
Using eos_token, but it is not set yet.
GSM8K_rewritten-train-3 5_gram_accuracy:  0.002866666666666667
no special token
100%|██████████| 3000/3000 [18:48<00:00,  2.66it/s]
Using eos_token, but it is not set yet.
Token indices sequence length is longer than the specified maximum sequence length for this model (592 > 512). Running this sequence through the model will result in indexing errors
GSM8K_rewritten-train-3 10_gram_accuracy:  0.0
3000
no special token
  8%|▊         | 236/3000 [00:48<09:26,  4.88it/s]This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (512). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.
100%|██████████| 3000/3000 [10:19<00:00,  4.84it/s]
Using eos_token, but it is not set yet.
MATH_rewritten-test-1 5_gram_accuracy:  0.003266666666666667
no special token
100%|██████████| 3000/3000 [18:51<00:00,  2.65it/s]
Using eos_token, but it is not set yet.
MATH_rewritten-test-1 10_gram_accuracy:  6.666666666666667e-05
3000
no special token
100%|██████████| 3000/3000 [10:11<00:00,  4.91it/s]
Using eos_token, but it is not set yet.
MATH_rewritten-test-2 5_gram_accuracy:  0.003733333333333334
no special token
100%|██████████| 3000/3000 [18:43<00:00,  2.67it/s]
Using eos_token, but it is not set yet.
MATH_rewritten-test-2 10_gram_accuracy:  6.666666666666667e-05
3000
no special token
100%|██████████| 3000/3000 [10:11<00:00,  4.90it/s]
Using eos_token, but it is not set yet.
MATH_rewritten-test-3 5_gram_accuracy:  0.003866666666666667
"""

results_5grm_summary = {}
results_10grm_summary = {}

# 使用正则表达式解析日志数据
for match in re.finditer(r'(GSM8K_rewritten-\w+-\d+|MATH_rewritten-\w+-\d+) (\d+_gram)_accuracy:\s+([\d\.]+)', log_data):
    dataset, gram, accuracy = match.groups()
    accuracy = float(accuracy)
    
    if gram == '5_gram':
        results_5grm_summary[dataset] = accuracy
    elif gram == '10_gram':
        results_10grm_summary[dataset] = accuracy

results_5grm_summary, results_10grm_summary

({'GSM8K_rewritten-test-1': 0.002577710386656558,
  'GSM8K_rewritten-test-2': 0.0027293404094010615,
  'GSM8K_rewritten-test-3': 0.003032600454890068,
  'GSM8K_rewritten-train-1': 0.0019333333333333336,
  'GSM8K_rewritten-train-2': 0.002266666666666667,
  'GSM8K_rewritten-train-3': 0.002866666666666667,
  'MATH_rewritten-test-1': 0.003266666666666667,
  'MATH_rewritten-test-2': 0.003733333333333334,
  'MATH_rewritten-test-3': 0.003866666666666667},
 {'GSM8K_rewritten-test-1': 0.0,
  'GSM8K_rewritten-test-2': 0.0,
  'GSM8K_rewritten-test-3': 0.0,
  'GSM8K_rewritten-train-1': 0.0,
  'GSM8K_rewritten-train-2': 6.666666666666667,
  'GSM8K_rewritten-train-3': 0.0,
  'MATH_rewritten-test-1': 6.666666666666667,
  'MATH_rewritten-test-2': 6.666666666666667})

### 将打印出来的md格式表格，去除“|---|--”一行后复制过来，生成excel文件（注意修改路径）

In [10]:
import pandas as pd
from io import StringIO
import openpyxl

# 定义Markdown表格数据
markdown_table = """
| model_type  | MATH-seen-rewritten-1 | MATH-seen-rewritten-2 | MATH-seen-rewritten-3 | MATH-seen-orgn | MATH-unseen-rewritten-1 | MATH-unseen-rewritten-2 | MATH-unseen-rewritten-3 | MATH-unseen-orgn |
| full_loss   | 3.82 | 3.56 | 3.55 | 2.41 | 4.49 | 4.25 | 4.26 | 3.78 |
| answer_loss | 4.18 | 3.98 | 3.97 | 3.16 | 4.69 | 4.48 | 4.49 | 4.13 |
| Mistral-7b  | 4.34 | 4.15 | 4.15 | 3.88 | 4.38 | 4.20 | 4.20 | 3.95 |
"""
# 修正每行的开头和结尾，以确保不会因为解析问题而创建未命名的列
cleaned_lines = []
for line in markdown_table.strip().split('\n'):
    cleaned_line = line.strip('|')  # 去除每行开头和结尾的'|'
    cleaned_lines.append(cleaned_line)
cleaned_markdown_table = '\n'.join(cleaned_lines)

# 使用StringIO读取清理后的数据
markdown_table_io = StringIO(cleaned_markdown_table)

# print(cleaned_markdown_table)

# 读取数据，这次不需要特别处理未命名列
df = pd.read_csv(markdown_table_io, sep='\s*\|\s*', engine='python')

# 指定一个绝对路径来保存Excel文件
output_path = 'C:\\Users\\26848\\Downloads\\meta-M-mink.xlsx'  # 替换为你想要保存文件的确切路径

# 保存DataFrame到Excel文件
df.to_excel(output_path, index=False)

print(f"Markdown表格（包括标题行）已成功保存为Excel文件 '{output_path}'")

df

,model_type,MATH-seen-rewritten-1,MATH-seen-rewritten-2,MATH-seen-rewritten-3,MATH-seen-orgn,MATH-unseen-rewritten-1,MATH-unseen-rewritten-2,MATH-unseen-rewritten-3,MATH-unseen-orgn
0,full_loss,3.82,3.56,3.55,2.41,4.49,4.25,4.26,3.78
1,answer_loss,4.18,3.98,3.97,3.16,4.69,4.48,4.49,4.13
2,Mistral-7b,4.34,4.15,4.15,3.88,4.38,4.20,4.20,3.95


### 绘制总实验结果图

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 14

# Read the Excel file
data = pd.read_excel(r"C:\Users\26848\Downloads\plot2.xlsx")

# Define the figure size and other properties for the plot
fig, ax = plt.subplots(figsize=(14, 6))

# Calculate the positions for the bars and the width
positions = np.arange(len(data))
width = 0.35

# Define the color scheme for the bars and lines
color_train = 'mediumseagreen'
color_test = 'darkslategray'

# Create the bars for Δ-train and Δ-test
ax.bar(positions - width/2, data['Δ-train'], width, label='Δ-train', color=color_train)
ax.bar(positions + width/2, data['Δ-test'], width, label='Δ-test', color=color_test)

ax2 = ax.twinx()

# Create the points for Δ%-train and Δ%-test
ax2.scatter(positions, data['Δ%-train'], color=color_train, zorder=5)
ax2.scatter(positions, data['Δ%-test'], color=color_test, zorder=5)

# Connect the points with lines
ax2.plot(positions, data['Δ%-train'], color=color_train, label='Δ%-train')
ax2.plot(positions, data['Δ%-test'], color=color_test, label='Δ%-test')

# Invert the y-axis so 0 is on top
ax.invert_yaxis()
ax2.invert_yaxis()

# Set the x-ticks and labels
ax.set_xticks(positions)
ax.set_xticklabels(data['avg_ppl(answer)'], rotation=45, ha='right')

# Align the labels with the center between the bars
for label, pos in zip(ax.get_xticklabels(), positions):
    label.set_x(pos - width/2)  # shift to the left by half the width of the bar to center

# Add labels, title, grid, and legend
# ax.set_xlabel('Models')
ax.set_ylabel('Decrement')
ax.set_title('Comparative Performance Decline: Original vs. Synthetic')
ax.grid(True)
# ax.legend()

# 创建图例的句柄
bar_seen_handle = mpatches.Patch(color=color_seen, label='$\Delta_{Train}$')
bar_unseen_handle = mpatches.Patch(color=color_unseen, label='$\Delta_{Test}$')
line_seen_handle = mlines.Line2D([], [], color=color_seen, marker='o', linestyle='-', label='$\hat{\Delta}_{Train}$')
line_unseen_handle = mlines.Line2D([], [], color=color_unseen, marker='o', linestyle='-', label='$\hat{\Delta}_{Test}$')

# 将它们组合到handles和labels列表中
handles = [bar_seen_handle, bar_unseen_handle, line_seen_handle, line_unseen_handle]
labels = [h.get_label() for h in handles]

plt.legend(handles, labels, loc='lower right', bbox_to_anchor=(1, 0))

# Tight layout to prevent clipping of tick labels
plt.tight_layout()

# Save the figure as a .png file
plt.savefig(r"C:\Users\26848\Downloads\main_results(ppl-G).pdf", format='pdf')

# Show the plot
plt.show()


### 绘制meta实验结果图

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.lines as mlines
import matplotlib.patches as mpatches

plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 17

# 读取Excel文件中所有工作表的名字
xls = pd.ExcelFile(r"C:\Users\26848\Downloads\meta-plots.xlsx")
sheet_names = xls.sheet_names

# 定义子图的布局：1行，列数等于工作表的数量
fig, axs = plt.subplots(1, len(sheet_names), figsize=(4*len(sheet_names), 6))

# 定义颜色方案
color_seen = 'mediumseagreen'
color_unseen = 'darkslategray'

# color_seen = '#4E95D9'
# color_unseen = '#8ED973'

# 设置比例尺
# 对于前两张图和后两张图的不同刻度设置
scale1 = [0, 3, 6, 9, 12]  # Δ-train 和 Δ-test 的刻度
scale2 = [0, 6, 12, 18, 24]   # Δ%-train 和 Δ%-test 的刻度
scale3 = [0, 0.2, 0.4, 0.6, 0.8]  # Δ-train 和 Δ-test 的刻度
scale4 = [0, 11, 22, 33, 44]   # Δ%-train 和 Δ%-test 的刻度

# 为每个工作表创建一个子图
for i, sheet_name in enumerate(sheet_names):
    data = pd.read_excel(xls, sheet_name=sheet_name)
    positions = np.arange(len(data))
    width = 0.35

    # 绘制条形图
    # axs[i].bar(positions - width/2, data['Δ-seen'], width, label='Δ-seen', color=color_seen, alpha=0.5)
    # axs[i].bar(positions + width/2, data['Δ-unseen'], width, label='Δ-unseen', color=color_unseen, alpha=0.5)
    
    # 绘制条形图
    axs[i].bar(positions - width/2, data['Δ-seen'], width, label='Δ-seen', color=color_seen)
    axs[i].bar(positions + width/2, data['Δ-unseen'], width, label='Δ-unseen', color=color_unseen)
    
    # 设置第一个和第二个y轴
    if i < 2:
        axs[i].set_yticks(scale1)
        ax2 = axs[i].twinx()
        ax2.set_ylim(0, max(scale2))  # 明确设置y轴显示范围
        ax2.set_yticks(scale2)
    else:
        axs[i].set_yticks(scale3)
        ax2 = axs[i].twinx()
        if i == 3:
            ax2.set_ylim(0, max(scale4))  # 明确设置y轴显示范围
        ax2.set_yticks(scale4)

    # 反转y轴
    axs[i].invert_yaxis()
    ax2.invert_yaxis()

    # 绘制散点图和线图
    ax2.scatter(positions, data['Δ%-seen'], color=color_seen, zorder=5)
    ax2.scatter(positions, data['Δ%-unseen'], color=color_unseen, zorder=5)
    ax2.plot(positions, data['Δ%-seen'], color=color_seen, label='Δ%-seen')
    ax2.plot(positions, data['Δ%-unseen'], color=color_unseen, label='Δ%-unseen')

    # 设置图表属性
    axs[i].set_xticks(positions)
    axs[i].set_xticklabels(data['Model Type'], rotation=45, ha='right')
    axs[i].set_title(sheet_name, fontsize=17)
    axs[i].grid(True)

# 创建图例的句柄
bar_seen_handle = mpatches.Patch(color=color_seen, label='$\Delta_{Seen}$')
bar_unseen_handle = mpatches.Patch(color=color_unseen, label='$\Delta_{Unseen}$')
line_seen_handle = mlines.Line2D([], [], color=color_seen, marker='o', linestyle='-', label='$\hat{\Delta}_{Seen}$')
line_unseen_handle = mlines.Line2D([], [], color=color_unseen, marker='o', linestyle='-', label='$\hat{\Delta}_{Unseen}$')

# 将它们组合到handles和labels列表中
handles = [bar_seen_handle, bar_unseen_handle, line_seen_handle, line_unseen_handle]
labels = [h.get_label() for h in handles]

# 设置整体图像的标题和轴标签
# fig.suptitle('Meta Experiment', fontsize=19, y=0.88)  # 调整y参数来向下移动suptitle
# fig.text(0.5, 0.125, 'Models', ha='center', va='center', fontsize=17)
fig.text(0.05, 0.5, 'Decrement', va='center', rotation='vertical', fontsize=17)

# 创建统一的图例并放在图片下方
fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, 0), ncol=len(handles), fontsize=17)

# 调整布局以腾出空间放置图例
plt.tight_layout(rect=[0.05, 0.07, 0.95, 0.95])

# Save the figure as a .png file
plt.savefig(r"C:\Users\26848\Downloads\meta_results.pdf", format='pdf')

# 显示图像
plt.show()

### 统计样本级别exact match结果

In [7]:
import json
import os

# Initialize variables
model_names = [
    "Aquila2-34B",
    "Aquila2-7B",
    "Baichuan2-13B-Base",
    "chatglm2-6b",
    "chatglm3-6b",
    "internlm2-20b",
    "internlm2-7b",
    "Orca-2-7b",
    "phi-1_5",
    "Qwen-1_8B",
    "Qwen-7B",
    "Qwen-14B",
    "Yi-6B",
    "Yi-34B"
]

dataset_names = [
    "MATH-test",
    "MATH-train",
    "GSM8K-test",
    "GSM8K-train"
]

results = {}

# Function to load JSON data and extract n_grams statistics
def load_and_process_file(path):
    with open(path, 'r') as file:
        data = json.load(file)
        ngram_counts = {str(i): 0 for i in [0, 0.2, 0.4, 0.6, 0.8, 1]}
        for item in data:
            n_grams = item['overall']['n_grams']
            for key in ngram_counts.keys():
                if n_grams == float(key):
                    ngram_counts[key] += 1
                    break
        return ngram_counts

# Processing each file and populating the results dictionary
for model_name in model_names:
    results[model_name] = {}
    for dataset_name in dataset_names:
        path = f"/home/rjxu/new-gram/results/5gram-{model_name}-{dataset_name}.json"
        if os.path.exists(path): # Ensure the file exists
            ngram_statistics = load_and_process_file(path)
            results[model_name][dataset_name] = ngram_statistics
        else:
            print(f"File not found: {path}")
            results[model_name][dataset_name] = {str(i): 0 for i in [0, 0.2, 0.4, 0.6, 0.8, 1]}

# Print results as a Markdown table
header = "| Model/Dataset | " + " | ".join([f"{dn}-nrgam={ng}" for dn in dataset_names for ng in [0, 0.2, 0.4, 0.6, 0.8, 1]]) + " |"
print(header)
print("|---" * (len(dataset_names) * 6 + 1) + "|")

for model_name, datasets in results.items():
    row = f"| {model_name} | "
    for dataset_name in dataset_names:
        for ng in [0, 0.2, 0.4, 0.6, 0.8, 1]:
            row += f"{datasets[dataset_name][str(ng)]} | "
    print(row)


| Model/Dataset | MATH-test-nrgam=0 | MATH-test-nrgam=0.2 | MATH-test-nrgam=0.4 | MATH-test-nrgam=0.6 | MATH-test-nrgam=0.8 | MATH-test-nrgam=1 | MATH-train-nrgam=0 | MATH-train-nrgam=0.2 | MATH-train-nrgam=0.4 | MATH-train-nrgam=0.6 | MATH-train-nrgam=0.8 | MATH-train-nrgam=1 | GSM8K-test-nrgam=0 | GSM8K-test-nrgam=0.2 | GSM8K-test-nrgam=0.4 | GSM8K-test-nrgam=0.6 | GSM8K-test-nrgam=0.8 | GSM8K-test-nrgam=1 | GSM8K-train-nrgam=0 | GSM8K-train-nrgam=0.2 | GSM8K-train-nrgam=0.4 | GSM8K-train-nrgam=0.6 | GSM8K-train-nrgam=0.8 | GSM8K-train-nrgam=1 |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Aquila2-34B | 229 | 729 | 964 | 708 | 363 | 7 | 1 | 18 | 249 | 1095 | 1611 | 26 | 317 | 579 | 381 | 42 | 0 | 0 | 86 | 777 | 1617 | 520 | 0 | 0 | 
| Aquila2-7B | 475 | 1014 | 924 | 455 | 132 | 0 | 104 | 529 | 926 | 949 | 486 | 6 | 499 | 584 | 215 | 21 | 0 | 0 | 330 | 1223 | 1276 | 171 | 0 | 0 | 
| Baichuan2-13B-Base | 366 | 988 | 976 | 547 |

### 统计样本级别rouge-l结果

In [4]:
import json
import os

# Initialize variables
model_names = [
    "Aquila2-34B",
    "Aquila2-7B",
    "Baichuan2-13B-Base",
    "chatglm2-6b",
    "chatglm3-6b",
    "internlm2-20b",
    "internlm2-7b",
    "Orca-2-7b",
    "phi-1_5",
    "Qwen-1_8B",
    "Qwen-7B",
    "Qwen-14B",
    "Yi-6B",
    "Yi-34B"
]

dataset_names = [
    "MATH-test",
    "MATH-train",
    "GSM8K-test",
    "GSM8K-train"
]

results = {}

# Function to load JSON data and extract n_grams statistics
def load_and_process_file(path):
    with open(path, 'r') as file:
        data = json.load(file)
        accuracies_rouge_l_counts = {str(i): 0 for i in [0, 0.2, 0.4, 0.6, 0.8, 1]}
        for item in data:
            accuracies_rouge_l = item['overall']['accuracies_rouge_l']
            for key in accuracies_rouge_l_counts.keys():
                if accuracies_rouge_l == float(key):
                    accuracies_rouge_l_counts[key] += 1
                    break
        return accuracies_rouge_l_counts

# Processing each file and populating the results dictionary
for model_name in model_names:
    results[model_name] = {}
    for dataset_name in dataset_names:
        path = f"/home/rjxu/new-gram/results/5gram-{model_name}-{dataset_name}.json"
        if os.path.exists(path): # Ensure the file exists
            accuracies_rouge_l_statistics = load_and_process_file(path)
            results[model_name][dataset_name] = accuracies_rouge_l_statistics
        else:
            print(f"File not found: {path}")
            results[model_name][dataset_name] = {str(i): 0 for i in [0, 0.2, 0.4, 0.6, 0.8, 1]}

# Print results as a Markdown table
header = "| Model/Dataset | " + " | ".join([f"{dn}-acc_rouge_l={ng}" for dn in dataset_names for ng in [0, 0.2, 0.4, 0.6, 0.8, 1]]) + " |"
print(header)
print("|---" * (len(dataset_names) * 6 + 1) + "|")

for model_name, datasets in results.items():
    row = f"| {model_name} | "
    for dataset_name in dataset_names:
        for ng in [0, 0.2, 0.4, 0.6, 0.8, 1]:
            row += f"{datasets[dataset_name][str(ng)]} | "
    print(row)


| Model/Dataset | MATH-test-acc_rouge_l=0 | MATH-test-acc_rouge_l=0.2 | MATH-test-acc_rouge_l=0.4 | MATH-test-acc_rouge_l=0.6 | MATH-test-acc_rouge_l=0.8 | MATH-test-acc_rouge_l=1 | MATH-train-acc_rouge_l=0 | MATH-train-acc_rouge_l=0.2 | MATH-train-acc_rouge_l=0.4 | MATH-train-acc_rouge_l=0.6 | MATH-train-acc_rouge_l=0.8 | MATH-train-acc_rouge_l=1 | GSM8K-test-acc_rouge_l=0 | GSM8K-test-acc_rouge_l=0.2 | GSM8K-test-acc_rouge_l=0.4 | GSM8K-test-acc_rouge_l=0.6 | GSM8K-test-acc_rouge_l=0.8 | GSM8K-test-acc_rouge_l=1 | GSM8K-train-acc_rouge_l=0 | GSM8K-train-acc_rouge_l=0.2 | GSM8K-train-acc_rouge_l=0.4 | GSM8K-train-acc_rouge_l=0.6 | GSM8K-train-acc_rouge_l=0.8 | GSM8K-train-acc_rouge_l=1 |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Aquila2-34B | 80 | 448 | 884 | 957 | 596 | 35 | 0 | 6 | 134 | 887 | 1872 | 101 | 146 | 490 | 527 | 155 | 1 | 0 | 29 | 446 | 1505 | 1013 | 7 | 0 | 
| Aquila2-7B | 204 | 726 | 1063 | 723 | 280 | 4 | 2

### 统计样本级别edit-distance结果

In [ ]:
import json
import os

# Initialize variables
model_names = [
    "Aquila2-34B",
    "Aquila2-7B",
    "Baichuan2-13B-Base",
    "chatglm2-6b",
    "chatglm3-6b",
    "internlm2-20b",
    "internlm2-7b",
    "Orca-2-7b",
    "phi-1_5",
    "Qwen-1_8B",
    "Qwen-7B",
    "Qwen-14B",
    "Yi-6B",
    "Yi-34B"
]

dataset_names = [
    "MATH-test",
    "MATH-train",
    "GSM8K-test",
    "GSM8K-train"
]

results = {}

# Function to load JSON data and extract n_grams statistics
def load_and_process_file(path):
    with open(path, 'r') as file:
        data = json.load(file)
        accuracies_edit_distance_counts = {str(i): 0 for i in [0, 0.2, 0.4, 0.6, 0.8, 1]}
        for item in data:
            accuracies_edit_distance = item['overall']['accuracies_edit_distance']
            for key in accuracies_edit_distance_counts.keys():
                if accuracies_edit_distance == float(key):
                    accuracies_edit_distance_counts[key] += 1
                    break
        return accuracies_edit_distance_counts

# Processing each file and populating the results dictionary
for model_name in model_names:
    results[model_name] = {}
    for dataset_name in dataset_names:
        path = f"/home/rjxu/new-gram/results/5gram-{model_name}-{dataset_name}.json"
        if os.path.exists(path): # Ensure the file exists
            accuracies_edit_distance_statistics = load_and_process_file(path)
            results[model_name][dataset_name] = accuracies_edit_distance_statistics
        else:
            print(f"File not found: {path}")
            results[model_name][dataset_name] = {str(i): 0 for i in [0, 0.2, 0.4, 0.6, 0.8, 1]}

# Print results as a Markdown table
header = "| Model/Dataset | " + " | ".join([f"{dn}-acc_edit_distance={ng}" for dn in dataset_names for ng in [0, 0.2, 0.4, 0.6, 0.8, 1]]) + " |"
print(header)
print("|---" * (len(dataset_names) * 6 + 1) + "|")

for model_name, datasets in results.items():
    row = f"| {model_name} | "
    for dataset_name in dataset_names:
        for ng in [0, 0.2, 0.4, 0.6, 0.8, 1]:
            row += f"{datasets[dataset_name][str(ng)]} | "
    print(row)


### 绘制样本级别统计图

In [6]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the excel file (Please change the path to where the file is located on your system)
excel_path = r"C:\Users\26848\Downloads\Instance-plot.xlsx"  # Replace with the correct path
excel_data = pd.ExcelFile(excel_path)

plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 17

# Read the contents of each sheet into a dictionary
sheets = {sheet: pd.read_excel(excel_data, sheet_name=sheet) for sheet in excel_data.sheet_names}

# Initialize a dictionary to hold the plot data
plot_data = {
    'N-gram': {},
    'Rouge-L': {}
}

# Define colors for each dataset
dataset_colors = {
    'MATH-train': '#4E95D9',
    'MATH-test': '#8ED973',
    'GSM8K-train': '#F2AA84',
    'GSM8K-test': '#E74C3C'
}

# Custom logarithmic scale
custom_log_scale = [1.0/3.0] + [1, 3, 9, 27, 81, 243, 729]
# custom_ticks_labels = ['$0$'] + ['$3^{0}$', '$3^{1}$', '$3^{2}$', '$3^{3}$', '$3^{4}$', '$3^{5}$', '$3^{6}$']
custom_ticks_labels = [0] + [1, 3, 9, 27, 81, 243, 729]

# Assign colors to models for plotting and organize data for N-gram and Rouge-L metrics
for sheet_name, df in sheets.items():
    df = df.set_index(df.columns[0])  # Use the first column as index (model names)
    for model_name in df.index:
        for col, metric in zip(df.columns, ['N-gram', 'Rouge-L']):
            if model_name not in plot_data[metric]:
                plot_data[metric][model_name] = []
            # Replace zeros with ones for log scale compatibility
            value = df.at[model_name, col] if df.at[model_name, col] != 0 else 1.0/3.0
            plot_data[metric][model_name].append(value)
            
# Model names are the x-axis labels
model_names = list(plot_data['N-gram'].keys())

# Create the plots
fig, axs = plt.subplots(1, 2, figsize=(15, 6))

# Plot each metric for each model
for metric, ax in zip(['N-gram', 'Rouge-L'], axs):
    for i, model in enumerate(model_names):
        if model == 'Models':  # Skip the 'Models' row
            continue
        for j, (dataset, color) in enumerate(dataset_colors.items()):
            ax.scatter(i, plot_data[metric][model][j], s=200, color=color, 
                       edgecolors=color, alpha=0.6, label=dataset if i == 0 else "")

# Setting titles, x and y labels
axs[0].set_title('Exact Match')
axs[1].set_title('Rouge-L > 0.75')
axs[0].set_xticks(range(len(model_names)))
axs[0].set_xticklabels(model_names, rotation=45, ha='right')
axs[1].set_xticks(range(len(model_names)))
axs[1].set_xticklabels(model_names, rotation=45, ha='right')

# Setting custom logarithmic scale for the y-axis
for ax in axs:
    ax.set_yscale('log')
    ax.set_ylim(bottom=1/3)  # Setting the bottom to 1/3 for the log scale
    ax.set_yticks(custom_log_scale)
    ax.set_yticklabels(custom_ticks_labels)
    ax.set_ylabel('Count', fontsize=18)

# Adding a single legend for both plots
legend_elements = [plt.Line2D([0], [0], marker='o', color=color, label=dataset, 
                              markersize=10, markerfacecolor=color, alpha=0.8)
                   for dataset, color in dataset_colors.items()]
fig.legend(handles=legend_elements, loc='lower center', bbox_to_anchor=(0.5, 0), ncol=4)

# Adjust layout to make room for the legend
plt.tight_layout(rect=[0, 0.05, 1, 0.95])

# Save the plot to a file
# plt.savefig(r"C:\Users\26848\Downloads\instance_results(num).pdf", format='pdf')

# Show the plot
plt.show()


ModuleNotFoundError: No module named 'matplotlib'